0. 추천시스템의 개념과 목적을 이해한다. \
1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다. \
2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다. \
3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다 \
4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.\
5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

In [4]:
import pandas as pd
import os

fname = 'C:/Users/minho/aiffel/datasets/lastfm-dataset-360K/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [5]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [6]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [7]:
condition = (data['user_id'] == data.loc[0, 'user_id'])
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [8]:
# 유저 수
data['user_id'].nunique()

358868

In [9]:
# 아티스트 수
data['artist'].nunique()

291346

In [10]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [11]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [12]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

In [13]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Greenday' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,Greenday,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


In [14]:
data[data.isin({'artist':['charlie']})['artist']]

,user_id,artist,play
703761,0a45d397a995abdac1c7a0a607348cd4dc72292b,charlie,5
1105860,1012f7870db6c6f7b304211533c13833680bf46d,charlie,62
1178157,112d3b62956be8d9ea554ca2c5271a617d1e74e3,charlie,531
1205756,11981a3e9d260ea0f6176936cdb32ab8a15c7f26,charlie,493
1264347,12721aa36c929e84b0d7584cde4782a75d69ea83,charlie,46
1589356,173166072c9783b949501f33e049931556e0a7ea,charlie,139
1736220,195aa9030703b88cea0f53499b253c25c40d0b7c,charlie,99
1771402,19da102f108cf3eb703b7060208d05339bdd7be3,charlie,255
1941757,1c5e9b1ee4a89b713a597138d2983d8d4dff3314,charlie,40
2010943,1d5563345c0c077c3d6dd249f5805acbaac12b1b,charlie,22


In [42]:
# data[data['artist'].str.contains('puth')]
# ValueError: Cannot mask with non-boolean array containing NA / NaN values
# (참고) pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

In [16]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['greenday' , 'maroon5' ,'jason mraz' ,'coldplay', 'boyslikegirls']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,Greenday,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


In [17]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v: k for k,v in enumerate(user_unique)}
artist_to_idx = {v: k for k,v in enumerate(artist_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,291347,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


In [20]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


In [24]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291348 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [26]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [27]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291348x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [29]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [30]:
zimin_vector

array([ 0.28931966,  0.13016461,  0.4820443 ,  0.8684773 ,  0.76990706,
       -0.01990928,  0.4320995 , -0.53518546, -0.27608168, -0.07631004,
       -0.5560449 , -0.1424984 , -0.23207568,  0.94252616,  0.6465114 ,
       -0.3883607 ,  0.9389626 ,  0.11596574,  0.5092602 , -0.302564  ,
        0.15649624, -0.42309257,  0.7122648 , -0.00860237,  0.4970869 ,
        0.9174501 , -1.2547638 ,  0.13766053,  0.70216316, -0.3102567 ,
        0.18553266, -0.6026885 ,  0.8664408 , -1.0409234 ,  0.46729   ,
        0.28021285,  0.4106776 , -0.29168233,  0.17307055,  0.07333227,
        0.73168015,  0.15174702, -0.5337671 ,  0.75291675,  0.81996375,
       -0.7728791 , -0.8644182 ,  1.0546411 , -0.37155697,  0.41670388,
       -0.81951874, -1.033974  ,  0.57654995,  0.18465365,  0.03241049,
       -0.9786494 ,  0.33166167,  0.77111286, -0.27280667, -0.05430903,
       -1.2720258 ,  0.6672439 , -0.48545736, -0.37305897, -0.9059694 ,
        0.1293865 , -0.9475728 , -0.15566199, -0.8743454 , -0.48

In [31]:
black_eyed_peas_vector

array([ 1.13441804e-02,  9.22240596e-03,  1.17513882e-02,  4.44753794e-03,
        1.59264170e-02,  1.37789436e-02,  1.59594137e-02, -1.55561101e-02,
        3.78945214e-03, -5.67360409e-03, -1.11159240e-03, -9.35640745e-03,
        7.66898878e-03,  8.77090904e-04,  2.90947035e-02,  2.80808774e-03,
        1.14121148e-03,  1.76311377e-02,  1.52827427e-02,  8.31579696e-03,
        1.43783651e-02,  1.09549088e-03,  2.15406734e-02,  2.54449365e-03,
       -4.43427591e-03,  1.38949323e-02, -1.89210121e-02,  1.85889360e-02,
        2.06668060e-02,  2.23152880e-02, -1.64005011e-02,  1.24868397e-02,
        1.13596348e-02,  1.66831315e-02,  1.99063811e-02,  1.89362150e-02,
        1.82862617e-02, -3.97738535e-04,  2.23321784e-02,  1.58121679e-02,
        2.09733918e-02,  2.95571430e-04, -1.21267363e-02,  2.01290045e-02,
        1.47742098e-02,  2.74772733e-03, -3.36646190e-05,  1.93836112e-02,
        5.36330976e-03, -4.37936420e-03,  1.74049614e-03,  1.32545978e-02,
        1.42027335e-02,  

In [32]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.30808058

In [33]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.2821554

In [34]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.99999994),
 (277, 0.9867383),
 (28, 0.97869253),
 (5, 0.97601587),
 (217, 0.97230047),
 (247, 0.9641383),
 (473, 0.9635477),
 (418, 0.9569228),
 (490, 0.9534205),
 (694, 0.9516821),
 (910, 0.95158476),
 (1018, 0.94428414),
 (782, 0.9440082),
 (268, 0.9432928),
 (531, 0.9377236)]

In [35]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'radiohead',
 'the beatles',
 'placebo',
 'u2',
 'oasis',
 'foo fighters',
 'nirvana',
 'the smashing pumpkins',
 'the white stripes',
 'pink floyd',
 'depeche mode']

In [36]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [37]:
get_similar_artist('2pac')

['2pac',
 'jay-z',
 'nas',
 'the game',
 'notorious b.i.g.',
 '50 cent',
 'dr. dre',
 'snoop dogg',
 'ludacris',
 'common']

In [38]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(409, 0.3565449),
 (2249, 0.3547741),
 (1800, 0.34847838),
 (550, 0.347241),
 (621, 0.34634292),
 (350, 0.34291273),
 (369, 0.3386866),
 (724, 0.3352223),
 (627, 0.32796484),
 (355, 0.32346857),
 (28, 0.31944042),
 (274, 0.31943968),
 (564, 0.31781545),
 (24, 0.31618837),
 (391, 0.3136029),
 (277, 0.31260145),
 (5, 0.3109454),
 (376, 0.30808055),
 (354, 0.30768314),
 (247, 0.30765495)]

In [39]:
[idx_to_artist[i[0]] for i in artist_recommended]

['amy winehouse',
 'katy perry',
 'lady gaga',
 'britney spears',
 'alicia keys',
 'rihanna',
 'justin timberlake',
 'lily allen',
 'maroon 5',
 'madonna',
 'the killers',
 'michael jackson',
 'kanye west',
 'jack johnson',
 'christina aguilera',
 'muse',
 'red hot chili peppers',
 'black eyed peas',
 'nelly furtado',
 'the beatles']

In [40]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)
explain

(0.3445636076966902,
 [(396, 0.24612083561721332),
  (3746, 0.05808046853589528),
  (62, 0.040600521462782285),
  (291347, -1.8116197219385883e-05),
  (270115, -0.00022010172198128113)],
 (array([[ 2.33499554,  2.34266725,  2.21829358, ...,  1.64622459,
           2.43523672,  2.51149701],
         [ 5.47011758,  1.29368905,  0.66597136, ...,  0.42955115,
           0.70750161,  0.77695006],
         [ 5.17970561,  6.05828358,  1.07699129, ...,  0.25811994,
           0.39271798,  0.46634607],
         ...,
         [ 3.84392706,  4.41226204,  4.21587112, ...,  0.87606928,
           0.00731687, -0.01348714],
         [ 5.68626687,  6.6202364 ,  6.29619964, ...,  4.62576184,
           0.84818948,  0.01066768],
         [ 5.86433431,  6.88873359,  6.59091484, ...,  4.75054031,
           7.1660503 ,  0.84551653]]),
  False))

In [41]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.24612083561721332),
 ('jason mraz', 0.05808046853589528),
 ('coldplay', 0.040600521462782285),
 ('Greenday', -1.8116197219385883e-05),
 ('maroon5', -0.00022010172198128113)]

In [ ]:
# 사실 저번주에 짠 코드인데 해야될게 기억이 안나서 이대로 올려요.....